<a href="https://colab.research.google.com/github/yun-feng/PrincipleCurve/blob/master/Bayesian_BMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import numpy as np

N=5
nrow=8
ncol=10

matrix=np.random.randint(2,size=(nrow,ncol,N))

rank_k=3

matrix_P=np.random.random(size=(nrow,rank_k))
matrix_Q=np.random.random(size=(rank_k,ncol))
prob=np.random.random()/2+0.5


def bounding(arr,epsilon=1e-6):
  arr[arr<epsilon]=epsilon
  arr[arr>1-epsilon]=1-epsilon
  return arr

def update_Z1(P,Q,prob):
  temp=1-P[:,:,np.newaxis]*Q[:,:]
  temp=temp.prod(1)
  z=(1-temp)*prob/((1-temp)*prob+temp*(1-prob))
  return bounding(z)

def update_Z2(P,Q,prob):
  temp=1-P[:,:,np.newaxis]*Q[:,:]
  temp=temp.prod(1)
  z=(1-temp)*(1-prob)/((1-temp)*(1-prob)+temp*prob)
  return bounding(z)

def update_prob(m,z1,z2):
  temp=m*z1[:,:,np.newaxis]+(1-m)*(1-z2)[:,:,np.newaxis]
  temp=temp.sum()
  temp/=N*nrow*ncol
  return max(temp,0.5)

def weight_cal(m,z1,z2):
  temp=m*z1[:,:,np.newaxis]+(1-m)*z2[:,:,np.newaxis]
  return temp

def update_u(P,Q):
  temp=1-P[:,:,np.newaxis]*Q[:,:]
  temp=np.log(temp)
  temp/=temp.sum(1)[:,np.newaxis,:]
  return bounding(temp)

def update_v(P,Q):
  temp=P[:,:,np.newaxis]*Q[:,:]
  temp=np.log(temp)
  temp=(np.log(P)[:,:,np.newaxis])/temp
  return bounding(temp)

def update_P(weight,u,v):
  #dim:nrow*rank*ncol*N
  temp1=weight[:,np.newaxis,:,:]*u[:,:,:,np.newaxis]
  temp2=(1-weight[:,np.newaxis,:,:])*v[:,:,:,np.newaxis]
  temp1=temp1.sum((2,3))
  temp2=temp2.sum((2,3))
  return bounding(temp1/(temp1+temp2))

def update_Q(weight,u,v):
  temp1=weight[:,np.newaxis,:,:]*u[:,:,:,np.newaxis]
  temp2=(1-weight[:,np.newaxis,:,:])*(1-v[:,:,:,np.newaxis])
  temp1=temp1.sum((0,3))
  temp2=temp2.sum((0,3))
  return bounding(temp1/(temp1+temp2))

def loss_cal(m,P,Q,prob):
  temp=1-P[:,:,np.newaxis]*Q[:,:]
  temp=temp.prod(1)
  prob1=np.log((1-temp)*prob+temp*(1-prob))
  prob0=np.log((1-temp)*(1-prob)+temp*prob)
  loss=-m*prob1[:,:,np.newaxis]-(1-m)*prob0[:,:,np.newaxis]
  loss=loss.sum()/loss.size
  return loss

old=loss_cal(matrix,matrix_P,matrix_Q,prob)
for i in range(100):
  #expectation step
  z1=update_Z1(matrix_P,matrix_Q,prob)
  z2=update_Z2(matrix_P,matrix_Q,prob)
  w=weight_cal(matrix,z1,z2)
  u=update_u(matrix_P,matrix_Q)
  v=update_v(matrix_P,matrix_Q)
  #maximization step
  prob=update_prob(matrix,z1,z2)
  matrix_P=update_P(w,u,v)
  matrix_Q=update_Q(w,u,v)
  
  loss=loss_cal(matrix,matrix_P,matrix_Q,prob)
  if old<loss:
    print(i,old,loss,old-loss)
    break
  old=loss
  print(loss)

print(matrix_P)
print(matrix_Q)

print(prob)


0.7032897125197104
0.6995109997203132
0.6977274994472518
0.696500446031305
0.6949505645066091
0.6934326686805206
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.6931471805599452
0.69314718055